In [86]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import torch
import networkx as nx
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import seaborn as sns
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster import hierarchy
import time

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dlpfc-151669/151669_qc_plot.png
/kaggle/input/dlpfc-151669/151669_count_matrix.csv
/kaggle/input/dlpfc-151669/151669_meta_data.csv


## Load metadata and count matrix from 12 samples

In [103]:
samples = {}
sample_numbers = ['151507', '151508', '151509', '151510', '151669', '151670', '151671', '151672', '151673', '151674', '151675', '151676']

for i in sample_numbers[0:2]:
    m_file = f"/kaggle/input/dlpfc-151669/{i}_meta_data.csv"  # Adjust for each sample's metadata file
    c_file = f"/kaggle/input/dlpfc-151669/{i}_count_matrix.csv"  # Adjust for each sample's count matrix file
    
    # Read the CSV files for metadata and count matrix
    m = pd.read_csv(m_file, index_col=0)
    c = pd.read_csv(c_file, index_col=0)
    
    # Store both the metadata and count matrix as a list for each sample
    samples[f'Sample {i}'] = [m, c]

# m1 = pd.read_csv("/kaggle/input/dlpfc-151669/151669_meta_data.csv", index_col =0) 
# c1 = pd.read_csv("/kaggle/input/dlpfc-151669/151669_count_matrix.csv", index_col =0)
# samples['Sample 1'] = [m1, c1]

# m2 = pd.read_csv("/kaggle/input/dlpfc-151669/151669_meta_data.csv", index_col =0) 
# c2 = pd.read_csv("/kaggle/input/dlpfc-151669/151669_count_matrix.csv", index_col =0)
# samples['Sample 1'] = [m2, c2]

## FUNCTION: Loop through 12 datasets and perform PCA, aggregation, and clustering with desired parameters. Save clustering plot and metrics.

In [105]:
# counts = samples['sample_1'][0] # Extract count matrix
# meta = samples['sample_1'][1] # Extract metadata

def run_samples(PCA_amount, num_hops, weight_weights, num_cluster):
    # Empty dictionary for clustering performance metrics
    metrics = {}
    # Iterate over each sample in dictionary
    for sample in samples:
        counts = samples[sample][1] # Extract count matrix
        meta = samples[sample][0] # Extract metadata
    
        # Remove the rows with NaN from both 'meta' and 'counts' DataFrames
        nan_index = meta[meta['Layer'].isna()].index
        meta = meta.drop(nan_index)
        counts = counts.drop(nan_index)
    
        # Convert ground truth cluster assignment to categorical
        meta['Layer'] = pd.Categorical(meta['Layer'])
    
        # Scale and PCA
        scaler = StandardScaler()
        count_scaled = scaler.fit_transform(counts)
        pca = PCA(n_components = PCA_amount)
        count_pca_array = pca.fit_transform(count_scaled)
        count_pca = pd.DataFrame(count_pca_array,index=counts.index)
    
        # Create graph object
        G = create_graph(sample, count_pca, meta)
        
        # Aggregate neighborhood features and concatenate with own features
        node_features = torch.from_numpy(count_pca_array)
        aggregated_features = aggregate_k_hops(G, node_features, num_hops, weight_weights)
        own_and_aggregated_features = torch.cat((node_features, aggregated_features), dim=1)
    
        # Cluster
        # kmeans = KMeans(n_clusters=5, init='random', random_state=42, n_init='auto')  # Specify the number of clusters
        # kmeans.fit(own_and_aggregated_features)
        # clusters = kmeans.labels_
        # meta['Clusters'] = clusters

        # Determine number of clusters using inertia elbow method, cluster using KMeans++
        k_values = range(5, 15)
        k , labels = best_k(k_values, own_and_aggregated_features, meta=None, figFile=f"{sample}: Elbow Plot.png")
        meta['Clusters'] = labels
    
        # Compute metrics and save to dict
        ari = adjusted_rand_score(meta['Layer'], clusters)
        metrics[sample] = [labels, ari, k]
    
        # Generate ground truth and clustering result spatial plots (colored by cluster)
        fig, axs = plt.subplots(1, 2, figsize=(12, 6))
        
        sns.scatterplot(x='X', y='Y', hue='Layer', data=meta, palette='Set1', s=10, ax=axs[0])
        axs[0].set_title(f"{sample}: Ground Truth")    
        
        sns.scatterplot(x='X', y='Y', hue='Clusters', data=meta, palette='Set1', s=10,ax=axs[1])
        axs[1].set_title(f"{sample}: Clusters")
        
        plt.tight_layout() # Adjust spacing between subplots
        plt.savefig(f'/kaggle/working/{sample}: Cluster Plot.png')
        plt.show()

    # Compute average and sd of metrics (ARI)
    metrics_matrix = np.array([value[1:] for value in metrics.values()])
    avg_ari = np.mean(metrics_matrix[:, 0])
    sd_ari = np.std(metrics_matrix[:, 0])
    
    return metrics, avg_ari, sd_ari

## FUNCTION: Create graph object from spatial coordinates and save plot

In [89]:
def create_graph(sample_name, count_pca, meta):
    num_spots = np.shape(count_pca)[0]  # number of spots (nodes)
    spatial_coordinates = meta.iloc[:,0:2]
 
    # Create graph objet
    G = nx.Graph()
     
    # Add nodes with their gene expression PC data as features
    for i in range(num_spots):
        G.add_node(i, features=torch.tensor(count_pca.iloc[i,], dtype=torch.float32))
     
    # Compute pairwise Euclidean distances between spots based on their spatial coordinates
    distance_matrix = cdist(spatial_coordinates, spatial_coordinates)
     
    # Define a distance threshold for creating edges based on spatial proximity
    distance_threshold = 150
     
    # Add edges based on the spatial distance threshold
    for i in range(num_spots):
        for j in range(i + 1, num_spots):
            if distance_matrix[i, j] < distance_threshold:
                G.add_edge(i, j)
    
    # Save node/edge plot    
    positions = {i: spatial_coordinates.iloc[i] for i in range(num_spots)}
    plt.figure(figsize=(10, 10)) 
    nx.draw(G, pos=positions, with_labels=False, node_size=10, node_color="red", font_size=12, font_weight="bold")
    plt.title(f"{sample_name}: Node and Edge Plot", fontsize=16)
     
    plt.savefig(f'/kaggle/working/{sample_name}: Node and Edge Plot.png')
    plt.show()

    return G

## FUNCTION: Aggregate neighborhood information from graph

In [83]:
def aggregate_k_hops(graph, node_features, k, weight_weights):
    weights = generate_exponential_decay_values(k) * weight_weights
    feature_dim = node_features.shape[1]
    aggregated_features = torch.zeros(node_features.shape[0], feature_dim * 1)
    
    for node in graph.nodes():
        current_neighbors = {node}
        all_neighbors = set()
        weighted_mean_sum = torch.zeros(feature_dim)
        feature_parts = []
        
        for hop in range(1, k + 1):
            # Expand to the next hop
            next_neighbors = set()
            for n in current_neighbors:
                next_neighbors.update(graph.neighbors(n))
            current_neighbors = next_neighbors - all_neighbors - {node}  # Avoid duplicates and self-loops
            all_neighbors.update(current_neighbors)
            
            # Aggregate features for the current hop
            if len(current_neighbors) > 0:
                neighbor_features = node_features[list(current_neighbors)]
                mean_features = neighbor_features.mean(dim=0) 
                # max_features = neighbor_features.max(dim=0).values
            else:
                mean_features = torch.zeros(feature_dim) 
                # max_features = torch.zeros(feature_dim)
            
            # feature_parts.extend([mean_features, max_features])
            # feature_parts.extend([mean_features])
            weighted_mean_sum += weights[hop - 1] * mean_features
        
        # Concatenate all aggregated features for this node
        # aggregated_features[node] = torch.cat(feature_parts)
        # aggregated_features[node] = torch.cat([weighted_mean_sum, max_features])
        aggregated_features[node] = weighted_mean_sum
    
    return aggregated_features


def generate_exponential_decay_values(k, rate=.5):
    # Generate k values that decay exponentially: e^(-rate * i)
    indices = np.arange(1, k + 1)  # Generate indices from 1 to k
    exp_values = np.exp(-rate * indices)  # Apply exponential decay
    
    # Normalize the values so that their sum equals 1
    normalized_values = exp_values / np.sum(exp_values)
    
    return normalized_values

## FUNCTION: Clustering

In [106]:
def find_elbow_point(k_values, costs):
    # Normalize k_values and costs to [0, 1] for comparison
    k_values_normalized = (k_values - np.min(k_values)) / (np.max(k_values) - np.min(k_values))
    costs_normalized = (costs - np.min(costs)) / (np.max(costs) - np.min(costs))
    # Calculate the difference from the line connecting first and last points
    line = np.array([k_values_normalized, costs_normalized]).T
    start, end = line[0], line[-1]
    distances = np.abs(np.cross(end-start, line-start) / np.linalg.norm(end-start))
    # The index with the maximum distance is the elbow point
    elbow_idx = np.argmax(distances)
    return k_values[elbow_idx], elbow_idx

def best_k(k_values, own_and_aggregated_features, meta=None, figFile="output.png"):
    inertia_values = []
    labels = []
 
    # Loop through the range of k values
    for i in k_values:
        kmeans = KMeans(n_clusters=i, init='k-means++', algorithm="lloyd", n_init='auto', random_state=0)
        kmeans.fit(own_and_aggregated_features)
        agg_labels = kmeans.labels_
        inertia_values.append(kmeans.inertia_)
        labels.append(agg_labels)
        if meta is not None:
            ari = adjusted_rand_score(meta['Layer'], agg_labels)
            print(f"k = {i}, Adjusted Rand Index (ARI): {ari}, inertia: {kmeans.inertia_}")
    # Plot the Elbow Method
    plt.figure(figsize=(10, 5))
    plt.plot(k_values, inertia_values, 'o-', label="Inertia (Elbow Method)")
    plt.xlabel("Number of Clusters (k)")
    plt.ylabel("Inertia")
    plt.title("Elbow Method")
    plt.legend()
    plt.grid()
    plt.savefig(figFile)
    # Determine the optimal k using the Elbow Method
    best_k, best_idx = find_elbow_point(k_values, inertia_values)
    print(f"The optimal k (Elbow Point) is: {best_k}")
    # Return the best k and the corresponding cluster labels
    return best_k, labels[best_idx]

## Perform hyperparameter search

In [ ]:
# Hyperparameter search values
hyp_PCA = [99, 90, 80]
hyp_num_hops = [1, 2, 3, 4, 5]
hyp_weight_weights = [.75, 1, 1.25, 1.5]

t = 0
for PCA_amount in hyp_PCA:
    for num_hops in hyp_num_hops:
        for weight_weights in hyp_weight_weights:
            t += 1

            # Run samples with specified hyperparameters
            start_time = time.time()
            metrics, avg_ari, sd_ari = run_samples(PCA_amount, num_hops, weight_weights, num_clusters)
            end_time = time.time()
            execution_time = end_time - start_time    

            print(t, execution_time, avg_ari, sd_ari, PCA_amount, num_hops, weight_weights)

In [ ]:
metrics, avg_ari, sd_ari = run_samples(PCA_amount=80, num_hops=3, weight_weights=1.25)

In [100]:
print(avg_ari, sd_ari)
print(metrics)

0.2502439102578752 0.0
{'sample_1': [array([1, 4, 3, ..., 1, 4, 2], dtype=int32), 0.2502439102578752]}
